# <a id='toc1_'></a>[set up](#toc0_)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
if sys.platform == 'linux':
    sys.path.append("/home/qix/MultiNeuronGLM")
else:
    sys.path.append("D:/Github/MultiNeuronGLM")

In [2]:
import pandas as pd
import utility_functions as utils
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np
import random
import torch
import logging
import joblib

import GLM
from DataLoader import Allen_dataset, Allen_dataloader_multi_session, BatchIterator

# sns.set_theme()
sns.set_theme(style="white")
# sns.set_style('whitegrid')

d:\conda_envs\allen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set random seed for reproducibility

random.seed(0)
np.random.seed(0) 
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)

logging.basicConfig(
    level=logging.WARNING,
    format='%(asctime)s - %(levelname)s - %(message)s - [%(filename)s:%(lineno)d]'
)

# Download all sessions

In [146]:
# Download all sessions
session_ids = [
    715093703, 719161530, 721123822, 732592105, 737581020, 739448407,
    742951821, 743475441, 744228101, 746083955, 750332458, 750749662,
    751348571, 754312389, 754829445, 755434585, 756029989, 757216464,
    757970808, 758798717, 759883607, 760345702, 760693773, 761418226,
    762120172, 762602078, 763673393, 766640955, 767871931, 768515987,
    771160300, 771990200, 773418906, 774875821, 778240327, 778998620,
    779839471, 781842082, 786091066, 787025148, 789848216, 791319847,
    793224716, 794812542, 797828357, 798911424, 799864342, 816200189,
    819186360, 819701982, 821695405, 829720705, 831882777, 835479236,
    839068429, 839557629, 840012044, 847657808
]
allen_dataset = cross_session_dataloader.sessions[715093703]
for session_id in tqdm(session_ids):
    allen_dataset._cache.get_session_data(session_id)


  0%|          | 0/58 [00:00<?, ?it/s]2025-01-21 00:47:07,398 - WARNING - downloading a 2723.916MiB file from http://api.brain-map.org//api/v2/well_known_file_download/1026124469 - [http_engine.py:73]
Downloading: 100%|██████████| 2.86G/2.86G [01:33<00:00, 30.6MB/s]
d:\conda_envs\allen\lib\site-packages\hdmf\spec\namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
d:\conda_envs\allen\lib\site-packages\hdmf\spec\namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
  2%|▏         | 1/58 [04:16<4:03:51, 256.69s/it]2025-01-21 00:48:43,836 - WARNING - downloading a 2929.156MiB file from http://api.brain-map.org//api/v2/well_known_file_download/1026124034 - [http_engine.py:73]
Do

# Load all sessions and save to hard drive

In [ ]:
# Build a dataloader for cross-session data
session_ids = [
    715093703, 719161530, 721123822, 732592105, 737581020, 739448407,
    742951821, 743475441, 744228101, 746083955, 750332458, 750749662,
    751348571, 754312389, 754829445, 755434585, 756029989, 757216464,
    757970808, 758798717, 759883607, 760345702, 760693773, 761418226,
    762120172, 762602078, 763673393, 766640955, 767871931, 768515987,
    771160300, 771990200, 773418906, 774875821, 778240327, 778998620,
    779839471, 781842082, 786091066, 787025148, 789848216, 791319847,
    793224716, 794812542, 797828357, 798911424, 799864342, 816200189,
    819186360, 819701982, 821695405, 829720705, 831882777, 835479236,
    839068429, 839557629, 840012044, 847657808
]
kwargs = {
    'shuffle':True,
    'align_stimulus_onset':False, 
    'merge_trials':False, 
    'batch_size':64,
    'fps':1000, 
    'start_time':0.0, 
    'end_time':0.4, 
    'padding':0.1, 
    'selected_probes':['probeA', 'probeB', 'probeC', 'probeD', 'probeE', 'probeF'], 
}
cross_session_dataloader = Allen_dataloader_multi_session(session_ids, **kwargs)

2025-01-21 18:39:59,998 - CRITICAL - Start loading data - [DataLoader.py:155]
  0%|          | 0/58 [00:00<?, ?it/s]d:\conda_envs\allen\lib\site-packages\hdmf\spec\namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
d:\conda_envs\allen\lib\site-packages\hdmf\spec\namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
100%|██████████| 58/58 [18:30<00:00, 19.15s/it]

Total sessions: 58, Batch size: 64, Train set size: 10178, Val set size: 1454, Test set size: 2908


In [ ]:
# Save the object
if sys.platform == 'linux':
    data_path = '/home/qix/user_data/allen_spike_trains/cross_session_dataloader.joblib'
else:
    data_path = 'D:/ecephys_cache_dir/cross_session_dataloader.joblib'
joblib.dump(cross_session_dataloader, data_path)

PicklingError: Can't pickle <class 'DataLoader.BatchIterator'>: it's not the same object as DataLoader.BatchIterator

# Reload from hard drive

In [4]:
# Load the object
if sys.platform == 'linux':
    data_path = '/home/qix/user_data/allen_spike_trains/cross_session_dataloader.joblib'
else:
    data_path = 'D:/ecephys_cache_dir/cross_session_dataloader.joblib'
cross_session_dataloader = joblib.load(data_path)

EOFError: 

In [75]:
for ibatch, batch in enumerate(cross_session_dataloader.train_loader):
    if ibatch == 9:
        break
    print(batch["session_id"])

739448407
831882777
774875821
831882777
831882777
847657808
759883607
762602078
760345702


In [55]:
batch

{'spike_trains': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 1],
         [0, 0, 0, ..., 1, 0, 0],
         [0, 0, 0, ..., 0, 0, 1],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[1, 0, 0, ..., 1, 1, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 0, 0],
         ...,
         [0, 0, 2, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 1]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 1],
         [2, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 1, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [1, 0, 0

In [56]:
batch.keys()

dict_keys(['spike_trains', 'presentation_ids', 'neuron_id', 'session_id'])

In [57]:
batch["spike_trains"].shape, batch["neuron_id"].shape, batch["presentation_ids"].shape, batch["session_id"]

((50, 166, 64), (166,), (64,), 719161530)

In [64]:
cross_session_dataloader.common_kwargs

{'align_stimulus_onset': False,
 'merge_trials': False,
 'fps': 1000,
 'start_time': 0.0,
 'end_time': 0.4,
 'padding': 0.1,
 'selected_probes': ['probeA',
  'probeB',
  'probeC',
  'probeD',
  'probeE',
  'probeF']}

In [58]:
cross_session_dataloader.sessions[715093703].fps

100.0

# Test how to get low resoluion spike trains faster

In [ ]:
# Control experiment: don't have low resolution but only original resolution
for ibatch, batch in enumerate(cross_session_dataloader.train_loader):
    if ibatch == 9:
        break
    # batch["low_res_spike_trains"] = utils.change_temporal_resolution_single(batch["spike_trains"], 10)

In [ ]:
# Get the low resolution spike trains afterwards
for ibatch, batch in enumerate(cross_session_dataloader.train_loader):
    if ibatch == 9:
        break
    batch["low_res_spike_trains"] = utils.change_temporal_resolution_single(batch["spike_trains"], 10)

In [ ]:
# Use np.histogram to bin the spike trains twice, once with the original resolution and once with the low resolution
for ibatch, batch in enumerate(cross_session_dataloader.train_loader):
    if ibatch == 9:
        break